In [33]:
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from datetime import datetime

In [23]:
# parser = argparse.ArgumentParser(description='VAE MNIST Example')
# parser.add_argument('--batch-size', type=int, default=128, metavar='N',
#                     help='input batch size for training (default: 128)')
# parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='enables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()

img_size = 96
batch_size=128
epochs=10
no_cuda=False
seed=1
log_interval=10
lin_size = img_size * img_size * 3
cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")

In [24]:
def load_data(data_dir, batch_size, shuffle=True, **kwargs):
    """ Method returning a data loader for labeled data """
    # TODO (optional): add data transformations if needed
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor()
        ]
    )
    data = datasets.ImageFolder(data_dir, transform=transform)
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=batch_size,
        shuffle=shuffle,
        **kwargs
    )
    return data_loader

In [25]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = load_data('/scratch/drr342/dl/project/data/ssl_data_96/supervised/train', batch_size)
test_loader = load_data('/scratch/drr342/dl/project/data/ssl_data_96/supervised/val', batch_size)

In [26]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(lin_size, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 27648)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, lin_size))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [27]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [28]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, lin_size), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [29]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [30]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, img_size, img_size)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [31]:
# if __name__ == "__main__":
#     for epoch in range(1, epochs + 1):
#         train(epoch)
#         test(epoch)
#         with torch.no_grad():
#             sample = torch.randn(64, 20).to(device)
#             sample = model.decode(sample).cpu()
#             save_image(sample.view(64, 1, img_size, img_size),
#                        'results/sample_' + str(epoch) + '.png')

In [36]:
print(f'/scratch/drr342/dl/project/models/resnet34.pth/{datetime.now().strftime('%Y%m%d_%H%M')}')

'20190502_1624'

In [32]:
for epoch in range(1, epochs + 1):
    train(epoch)
#     test(epoch)
#     with torch.no_grad():
#         sample = torch.randn(64, 20).to(device)
#         sample = model.decode(sample).cpu()
#         save_image(sample.view(64, 1, img_size, img_size),
#                    'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/64000 (0%)]	Loss: 19386.312500
Train Epoch: 1 [1280/64000 (2%)]	Loss: 19701.714844
Train Epoch: 1 [2560/64000 (4%)]	Loss: 19204.814453
Train Epoch: 1 [3840/64000 (6%)]	Loss: 18794.130859
Train Epoch: 1 [5120/64000 (8%)]	Loss: 18201.404297
Train Epoch: 1 [6400/64000 (10%)]	Loss: 18152.906250
Train Epoch: 1 [7680/64000 (12%)]	Loss: 17976.531250
Train Epoch: 1 [8960/64000 (14%)]	Loss: 17865.169922
Train Epoch: 1 [10240/64000 (16%)]	Loss: 17971.666016
Train Epoch: 1 [11520/64000 (18%)]	Loss: 17587.794922
Train Epoch: 1 [12800/64000 (20%)]	Loss: 17531.412109
Train Epoch: 1 [14080/64000 (22%)]	Loss: 17212.566406
Train Epoch: 1 [15360/64000 (24%)]	Loss: 17304.423828
Train Epoch: 1 [16640/64000 (26%)]	Loss: 17061.835938
Train Epoch: 1 [17920/64000 (28%)]	Loss: 16946.871094
Train Epoch: 1 [19200/64000 (30%)]	Loss: 17180.861328
Train Epoch: 1 [20480/64000 (32%)]	Loss: 16962.814453
Train Epoch: 1 [21760/64000 (34%)]	Loss: 16861.031250
Train Epoch: 1 [23040/64000 (36%)]	Loss: 168

KeyboardInterrupt: 